# Exploratory Yelp Data Analysis and Visualization 

## Group: 

**Notes: this cell is reserved as a temperary Table of contents, please put any add-functionalities to this main notebook here**  
Table of contents  
Motivations  
Which cities to crawl (from two resources)  
Explain the Scrapy framework and functions

## Table of Contents:
* [Project Overview](#first-bullet)
   - [Part I: Data Loading and Pre-Processing](#sub1)
   - [Part II: Feature Selection & Engineering](#second-bullet)
    - [Section 2.1: Feature Selection Principle](#sub1)
      - [Features to be dropped](#sub2)
      - [Features to be re-engineered](#sub2)
      - [Features to be kept even there are missing, special imputation needed](#sub2)
    - [Section 2.2: Data Imputation](#sub1)
      - [Imputation using fillna to 0](#sub2)
      - [sklearn Imputation using the mean](#sub2)
    - [Section 2.3: Data Normalization](#sub1)
    - [Section 2.4: Data Spliting](#sub1)
   - [Part III: Classification Model Building](#third-bullet)
    - [Section 1: Selection of the Methodology](#sub1)
    - [Section 2: Method Implementation and Pipeline](#sub1)
   - [Part IV: Classification Model Optimization](#fourth-bullet)
   - [Part V: Extracting Feature Importance](#fifth-bullet)
   - [Part VI: Multi-class exploration](#fifth-bullet)
* [Comment on Future work](#future-work)

## Project Overview and Motivations
The 

In [20]:
import pandas as pd
import sys
import numpy as np
# from pathlib import Path
sys.path.insert(0, '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
import pymysql

In [2]:
# helper functions from Codebase 
sys.path.insert(0, '../Codebase/')
from db_utils import YelpDb

## Read data from sql server into pandas

**Note**: this method is only for reading dumped .sql file that already live in local server; for usage of that .sql file on another machine, you need to refer this:  
https://blog.csdn.net/spartanfuk/article/details/81368465?depth_1-utm_source=distribute.pc_relevant.none-task

**Workflow**: 
1. .sql files to DF.
2. DFs to csv.
3. csv. to EDA
4. delete csv.


In [3]:
# playground


### Show all tables in two databases 

In [4]:
# build DB connections
yelp_db_group_1 = YelpDb("yelp_db_1_6")  # cities 1-6 (group 1)
yelp_db_group_2 = YelpDb("yelp_db")  # cities 7-12 (group 2)

sql_tabs = "show TABLES;"
yelp_db_group_1.execute(sql_tabs)
yelp_db_group_1.fetch_all()

(('yelp_fresno',),
 ('yelp_los_angeles',),
 ('yelp_sacramento',),
 ('yelp_san_diego',),
 ('yelp_san_francisco',),
 ('yelp_san_jose',))

In [5]:
yelp_db_group_2.execute(sql_tabs)
yelp_db_group_2.fetch_all()

(('yelp_Anaheim',),
 ('yelp_Bakersfield',),
 ('yelp_Long_Beach',),
 ('yelp_Oakland',),
 ('yelp_Riverside',),
 ('yelp_Santa_Ana',))

### Read databases into DataFrame 

In [6]:
# Table 1
yelp_df_la = yelp_db_group_1.db_to_df("yelp_los_angeles")
print(f'Yelp data for Los Angeles: number of rows (restaurants) = {yelp_df_la.shape[0]}')
print(f'number of columns (features) = {yelp_df_la.shape[1]}')
yelp_df_la.head(3)

Yelp data for Los Angeles: number of rows (restaurants) = 990
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Dave's Hot Chicken,"3462 Wilshire Blvd,Los Angeles,CA,90010",Chicken Shop,$11-30,4.0,82,11:00 am - 1:00 am,11:00 am - 1:00 am,11:00 am - 1:00 am,11:00 am - 1:00 am,...,11:00 am - 1:00 am,No,No,No,"Garage, Street",NULL,Yes,Yes,No,Yes
1,Chef Kang Sul Box,"3881 Wilshire Blvd,Los Angeles,CA,90010","Korean,Gastropubs,Chicken Shop",$11-30,5.0,31,5:00 pm - 2:00 am,5:00 pm - 2:00 am,5:00 pm - 2:00 am,5:00 pm - 2:00 am,...,5:00 pm - 2:00 am,Yes,Free,Yes,"Street, Private Lot, Validated",NULL,Yes,NULL,NULL,Yes
2,CHD,"3377 Wilshire Blvd,Ste 104,Los Angeles,CA,90010",Korean,$11-30,4.5,26,11:00 am - 8:30 pm,11:00 am - 8:30 pm,11:00 am - 8:30 pm,11:00 am - 8:30 pm,...,Closed,No,No,No,"Valet, Validated",Yes,Yes,No,NULL,Yes


In [7]:
# Table 2
yelp_df_sd = yelp_db_group_1.db_to_df("yelp_san_diego")
print(f'Yelp data for San Diego: number of rows (restaurants) = {yelp_df_sd.shape[0]}')
print(f'number of columns (features) = {yelp_df_sd.shape[1]}')
yelp_df_sd.head(3)

Yelp data for San Diego: number of rows (restaurants) = 990
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Liberty Public Market,"2820 Historic Decatur Rd,San Diego,CA,92106","Farmers Market,Food Court,Public Markets",$11-30,4.0,927,10:30 am - 9:00 pm,10:30 am - 9:00 pm,10:30 am - 9:00 pm,10:30 am - 9:00 pm,...,10:30 am - 9:00 pm,No,Free,Yes,"Street, Private Lot",NULL,Yes,Yes,Yes,Yes
1,Bud & Rob's New Orleans Bistro,"815 F St,San Diego,CA,92101","Cajun/Creole,Sandwiches,Desserts",$11-30,5.0,467,Closed,11:00 am - 2:30 pm,11:00 am - 2:30 pm,11:00 am - 8:00 pm,...,Closed,Yes,Free,Yes,Street,Yes,Yes,No,No,Yes
2,Barbusa,"1917 India St,San Diego,CA,92101","Sicilian,Seafood,Pizza",$11-30,4.0,2002,11:30 am - 10:00 pm,11:30 am - 10:00 pm,11:30 am - 10:00 pm,11:30 am - 10:00 pm,...,11:30 am - 10:00 pm,No,Free,Yes,"Street, Private Lot",Yes,Yes,No,No,Yes


In [8]:
# Table 3
yelp_df_sj = yelp_db_group_1.db_to_df("yelp_san_jose")
print(f'Yelp data for San Jose: number of rows (restaurants) = {yelp_df_sj.shape[0]}')
print(f'number of columns (features) = {yelp_df_sj.shape[1]}')
yelp_df_sj.head(3)

Yelp data for San Jose: number of rows (restaurants) = 990
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Firehouse No.1 Gastropub,"69 N San Pedro St,San Jose,CA,95110","American (Traditional),Pubs,Gastropubs",$11-30,4.0,1588,11:30 am - 12:30 am,11:30 am - 12:30 am,11:30 am - 12:30 am,11:30 am - 12:30 am,...,11:30 am - 12:30 am,No,Free,Yes,"Garage, Street, Validated",NULL,Yes,No,No,Yes
1,Élyse Restaurant,"151 S 2nd St,San Jose,CA,95113","French,Vietnamese,Cocktail Bars",$11-30,4.5,285,5:00 pm - 9:00 pm,5:00 pm - 9:00 pm,5:00 pm - 9:00 pm,5:00 pm - 9:00 pm,...,Closed,No,No,Yes,NULL,Yes,Yes,No,No,Yes
2,Hapa Musubi,"200 S 1st St,Unit 50,San Jose,CA,95112","Hawaiian,Japanese",Under $10,4.5,98,11:00 am - 4:30 pm,11:00 am - 4:30 pm,11:00 am - 4:30 pm,11:00 am - 4:30 pm,...,Closed,No,NULL,No,"Garage, Street, Validated",NULL,Yes,Yes,NULL,Yes


In [9]:
# Table 4
yelp_df_sf = yelp_db_group_1.db_to_df("yelp_san_francisco")
print(f'Yelp data for San Francisco: number of rows (restaurants) = {yelp_df_sf.shape[0]}')
print(f'number of columns (features) = {yelp_df_sf.shape[1]}')
yelp_df_sf.head(3)

Yelp data for San Francisco: number of rows (restaurants) = 990
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Tadu Ethiopian Kitchen,"1120-30 4th St,San Francisco,CA,94158",Ethiopian,$11-30,4.5,249,11:30 am - 9:30 pm,11:30 am - 9:30 pm,11:30 am - 9:30 pm,11:30 am - 9:30 pm,...,11:00 am - 9:30 pm,No,No,Yes,Street,Yes,Yes,Yes,Yes,Yes
1,Noosh,"2001 Fillmore St,San Francisco,CA,94115",Mediterranean,$11-30,3.5,325,11:00 am - 10:00 pm,11:00 am - 10:00 pm,11:00 am - 10:00 pm,11:00 am - 10:00 pm,...,11:00 am - 9:00 pm,Yes,Free,No,Street,NULL,Yes,NULL,NULL,Yes
2,Loló,"974 Valencia St,San Francisco,CA,94110","Mexican,Tapas/Small Plates",$11-30,4.0,2194,5:30 pm - 11:00 pm,5:30 pm - 11:00 pm,5:30 pm - 11:00 pm,5:30 pm - 11:00 pm,...,11:00 am - 4:00 pm,No,No,Yes,Street,NULL,Yes,No,Yes,No


In [10]:
# Table 5
yelp_df_fr = yelp_db_group_1.db_to_df("yelp_fresno")
print(f'Yelp data for Fresno: number of rows (restaurants) = {yelp_df_fr.shape[0]}')
print(f'number of columns (features) = {yelp_df_fr.shape[1]}')
yelp_df_fr.head(3)

Yelp data for Fresno: number of rows (restaurants) = 989
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Angry Chickz,"5044 N West Ave,Fresno,CA,93711","Chicken Shop,American (Traditional),Southern",$11-30,4.5,137,11:00 am - 11:00 pm,11:00 am - 11:00 pm,11:00 am - 11:00 pm,11:00 am - 11:00 pm,...,11:00 am - 11:00 pm,No,No,No,Street,NULL,Yes,Yes,NULL,Yes
1,SabaiDee Thai Lao Cuisine,"5730 N First St,Ste 103,Fresno,CA,93710","Laotian,Thai,Seafood",$11-30,4.0,643,11:00 am - 9:00 pm,11:00 am - 9:00 pm,11:00 am - 9:00 pm,11:00 am - 9:00 pm,...,Closed,No,No,Yes,Private Lot,Yes,Yes,No,No,Yes
2,Noodle Q Home Style Fresh Noodles,"2648 E Ashlan Ave,Fresno,CA,93726","Chinese,Soup,Noodles",Under $10,4.0,560,11:00 am - 9:00 pm,11:00 am - 9:00 pm,Closed,11:00 am - 9:00 pm,...,11:00 am - 9:00 pm,No,Free,Yes,Private Lot,Yes,Yes,No,No,Yes


In [11]:
# Table 6
yelp_df_sc = yelp_db_group_1.db_to_df("yelp_sacramento")
print(f'Yelp data for Sacramento: number of rows (restaurants) = {yelp_df_sc.shape[0]}')
print(f'number of columns (features) = {yelp_df_sc.shape[1]}')
yelp_df_sc.head(3)

Yelp data for Sacramento: number of rows (restaurants) = 990
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Don Chuy's Birria,"6035 Stockton Blvd,Sacramento,CA,95826",Mexican,Under $10,4.5,156,Closed,Closed,10:00 am - 6:00 pm,10:00 am - 6:00 pm,...,9:00 am - 3:00 pm,No,No,No,Street,NULL,No,No,No,Yes
1,Tako Korean BBQ,"3030 T St,Sacramento,CA,95816","Barbeque,Korean,Tacos",Under $10,4.0,1304,10:30 am - 10:00 pm,10:30 am - 10:00 pm,10:30 am - 10:00 pm,10:30 am - 10:00 pm,...,10:30 am - 10:00 pm,No,No,No,"Garage, Street, Private Lot",NULL,Yes,Yes,No,Yes
2,Hao Bao Dumplings,"6821 Stockton Blvd,Ste 100,Sacramento,CA,95823","Chinese,Noodles,Soup",Under $10,4.5,311,10:30 am - 8:00 pm,10:30 am - 8:00 pm,10:30 am - 8:00 pm,10:30 am - 8:00 pm,...,10:30 am - 8:00 pm,Yes,No,No,Private Lot,Yes,Yes,Yes,Yes,Yes


In [12]:
# Table 7
yelp_df_lb = yelp_db_group_2.db_to_df("yelp_Long_Beach")
print(f'Yelp data for Long Beach: number of rows (restaurants) = {yelp_df_lb.shape[0]}')
print(f'number of columns (features) = {yelp_df_lb.shape[1]}')
yelp_df_lb.head(3)

Yelp data for Long Beach: number of rows (restaurants) = 989
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,FB Nashville Hot Chicken,"3630 Atlantic Ave,Liberation Brewing,Long Beac...",Chicken Shop,$11-30,4.5,51,Closed,Closed,4:00 pm - 10:00 pm,4:00 pm - 10:00 pm,...,3:00 pm - 9:00 pm,Yes,Free,No,"Street, Private Lot",NULL,Yes,NULL,NULL,Yes
1,Gus's World Famous Fried Chicken,"2580 Long Beach Blvd,Long Beach,CA,90806","Southern,Chicken Shop,Comfort Food",$11-30,4.0,657,11:00 am - 9:00 pm,11:00 am - 9:00 pm,11:00 am - 9:00 pm,11:00 am - 9:00 pm,...,11:00 am - 9:00 pm,Yes,No,No,"Street, Private Lot",NULL,Yes,Yes,No,Yes
2,Nick's on 2nd,"4901 E 2nd St,Long Beach,CA,90803","American (New),Breakfast & Brunch,Steakhouses",$11-30,4.5,2185,11:00 am - 10:00 pm,11:00 am - 10:00 pm,11:00 am - 10:00 pm,11:00 am - 11:00 pm,...,8:00 am - 10:00 pm,No,No,Yes,"Garage, Street, Private Lot",NULL,Yes,No,No,Yes


In [13]:
# Table 8
yelp_df_ok = yelp_db_group_2.db_to_df("yelp_Oakland")
print(f'Yelp data for Oakland: number of rows (restaurants) = {yelp_df_ok.shape[0]}')
print(f'number of columns (features) = {yelp_df_ok.shape[1]}')
yelp_df_ok.head(3)

Yelp data for Oakland: number of rows (restaurants) = 990
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Brenda's Oakland,"4045 Broadway,Oakland,CA,94611","Breakfast & Brunch,Southern,Cajun/Creole",$11-30,4.0,305,8:00 am - 9:00 pm,Closed,8:00 am - 9:00 pm,8:00 am - 9:00 pm,...,8:00 am - 9:00 pm,Yes,No,No,Street,Yes,Yes,Yes,Yes,Yes
1,La Perla,"2020 MacArthur Blvd,Oakland,CA,94602",Puerto Rican,$11-30,4.5,156,Closed,Closed,12:00 pm - 7:00 pm,12:00 pm - 7:00 pm,...,12:00 pm - 5:00 pm,Yes,Free,No,Private Lot,NULL,Yes,Yes,NULL,Yes
2,MAMA Oakland,"388 Grand Ave,Oakland,CA,94610","Italian,Wine Bars",$31-60,4.0,102,5:00 pm - 10:00 pm,5:00 pm - 10:00 pm,5:00 pm - 10:00 pm,5:00 pm - 10:00 pm,...,5:00 pm - 10:00 pm,No,No,Yes,NULL,NULL,Yes,NULL,NULL,No


In [14]:
# Table 9
yelp_df_bf = yelp_db_group_2.db_to_df("yelp_Bakersfield")
print(f'Yelp data for Bakersfield: number of rows (restaurants) = {yelp_df_bf.shape[0]}')
print(f'number of columns (features) = {yelp_df_bf.shape[1]}')
yelp_df_bf.head(3)

Yelp data for Bakersfield: number of rows (restaurants) = 989
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Kalbi King Korean BBQ & Sushi,"7800 McNair Ln,Ste A,Bakersfield,CA,93313","Korean,Sushi Bars,Barbeque",$11-30,4.0,214,11:00 am - 10:00 pm,11:00 am - 10:00 pm,11:00 am - 10:00 pm,11:00 am - 10:00 pm,...,11:00 am - 10:00 pm,No,Free,Yes,"Street, Private Lot",NULL,Yes,NULL,NULL,Yes
1,Claw Daddy's Cajun Seafood,"2800 Taft Hwy,Bakersfield,CA,93313","Seafood,Cajun/Creole",$11-30,4.5,184,Closed,Closed,2:00 pm - 9:00 pm,2:00 pm - 9:00 pm,...,12:00 pm - 9:00 pm,No,No,Yes,Private Lot,NULL,Yes,Yes,NULL,Yes
2,Bootleggers Craft Pub & Eatery,"955 Oak St,Bakersfield,CA,93304","American (New),Pubs",$11-30,4.0,434,11:00 am - 10:30 pm,11:00 am - 10:30 pm,11:00 am - 10:30 pm,11:00 am - 10:30 pm,...,11:00 am - 10:30 pm,No,Free,No,Private Lot,Yes,Yes,No,Yes,Yes


In [15]:
# Table 10
yelp_df_ah = yelp_db_group_2.db_to_df("yelp_Anaheim")
print(f'Yelp data for Anaheim: number of rows (restaurants) = {yelp_df_ah.shape[0]}')
print(f'number of columns (features) = {yelp_df_ah.shape[1]}')
yelp_df_ah.head(3)

Yelp data for Anaheim: number of rows (restaurants) = 984
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Pho Ha Plus,"2191 S Harbor Blvd,Anaheim,CA,92802",Vietnamese,$11-30,4.0,340,11:00 am - 2:00 am,11:00 am - 2:00 am,11:00 am - 2:00 am,11:00 am - 2:00 am,...,11:00 am - 2:00 am,No,Free,No,Private Lot,Yes,Yes,Yes,NULL,Yes
1,The Blind Rabbit,"440 S Anaheim Blvd,Ste 104,Anaheim,CA,92805","Gastropubs,Speakeasies",$11-30,4.5,2031,5:00 pm - 11:00 pm,5:00 pm - 12:00 am,5:00 pm - 12:00 am,5:00 pm - 12:00 am,...,"12:00 pm - 4:00 pm,5:00 pm - 11:00 pm",No,No,Yes,"Valet, Garage, Street",NULL,Yes,No,No,No
2,PokiNometry,"184 S Harbor Blvd,Anaheim,CA,92805","Japanese,Seafood,Poke",Under $10,4.0,2882,11:00 am - 9:00 pm,11:00 am - 9:00 pm,11:00 am - 9:00 pm,11:00 am - 9:00 pm,...,11:00 am - 9:00 pm,No,Free,No,Street,NULL,Yes,Yes,No,Yes


In [16]:
# Table 11
yelp_df_sa = yelp_db_group_2.db_to_df("yelp_Santa_Ana")
print(f'Yelp data for Santa_Ana: number of rows (restaurants) = {yelp_df_sa.shape[0]}')
print(f'number of columns (features) = {yelp_df_sa.shape[1]}')
yelp_df_sa.head(3)

Yelp data for Santa_Ana: number of rows (restaurants) = 990
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Benchmark,"601 E Santa Ana Blvd,Santa Ana,CA,92701",American (New),$11-30,4.5,403,Closed,11:00 am - 10:00 pm,11:00 am - 10:00 pm,11:00 am - 10:00 pm,...,11:00 am - 10:00 pm,No,Free,Yes,"Street, Private Lot",Yes,Yes,NULL,NULL,Yes
1,Navarros Taqueria,"1535 S Standard Ave,Santa Ana,CA,92707",Tacos,Under $10,4.5,126,10:00 am - 9:00 pm,10:00 am - 9:00 pm,10:00 am - 9:00 pm,10:00 am - 9:00 pm,...,9:00 am - 10:00 pm,No,No,No,Private Lot,NULL,Yes,NULL,NULL,Yes
2,I Can Barbecue Santa Ana Korean Grill,"414 W 4th St,Santa Ana,CA,92701","Korean,Barbeque",$11-30,4.5,172,11:00 am - 11:00 pm,11:00 am - 11:00 pm,11:00 am - 11:00 pm,11:00 am - 11:00 pm,...,11:00 am - 11:00 pm,No,Free,Yes,Street,NULL,Yes,Yes,NULL,No


In [17]:
# Table 12
yelp_df_rs = yelp_db_group_2.db_to_df("yelp_Riverside")
print(f'Yelp data for Riverside: number of rows (restaurants) = {yelp_df_rs.shape[0]}')
print(f'number of columns (features) = {yelp_df_rs.shape[1]}')
yelp_df_rs.head(3)

Yelp data for Riverside: number of rows (restaurants) = 988
number of columns (features) = 22


,Name,Address,Category,Price,Rating,Reviews,Mon,Tue,Wed,Thu,...,Sun,Delivery,Wi_Fi,Takes_Reservations,Parking,Vegetarian_Options,Accepts_Credit_Cards,Accepts_Apple_Pay,Accepts_Google_Pay,Take_out
0,Hao pot,"1242 University Ave,Unit 5,Riverside,CA,92507","Asian Fusion,Chinese",$11-30,5.0,5,,,,,...,,Yes,Free,No,Private Lot,Yes,Yes,No,NULL,Yes
1,Espettos Grill,"5225 Canyon Crest Dr,Ste 57,Riverside,CA,92507",Brazilian,$11-30,4.5,186,Closed,11:00 am - 9:00 pm,11:00 am - 9:00 pm,11:00 am - 9:00 pm,...,11:00 am - 7:00 pm,Yes,Free,No,NULL,NULL,Yes,Yes,NULL,Yes
2,Kabob House,"10901 Magnolia Ave,Riverside,CA,92505","Mediterranean,Halal,Kebab",$11-30,4.5,538,10:30 am - 9:00 pm,10:30 am - 9:00 pm,10:30 am - 9:00 pm,10:30 am - 9:00 pm,...,11:00 am - 8:00 pm,Yes,No,No,Private Lot,NULL,Yes,Yes,No,Yes


### Temp: export as csv files to ../Database/temp_csv/

In [18]:
"""
yelp_df_la.to_csv(r'../Database/temp_csv/Los_Angeles.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_sd.to_csv(r'../Database/temp_csv/San_Diego.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_sj.to_csv(r'../Database/temp_csv/San_Jose.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_sf.to_csv(r'../Database/temp_csv/San_Francisco.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_fr.to_csv(r'../Database/temp_csv/Fresno.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_sc.to_csv(r'../Database/temp_csv/Sacramento.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_lb.to_csv(r'../Database/temp_csv/Long_Beach.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_ok.to_csv(r'../Database/temp_csv/Oakland.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_bf.to_csv(r'../Database/temp_csv/Bakersfield.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_ah.to_csv(r'../Database/temp_csv/Anaheim.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_sa.to_csv(r'../Database/temp_csv/Santa_Ana.csv', sep='\t', encoding='utf-8', header='true')
yelp_df_rs.to_csv(r'../Database/temp_csv/Riverside.csv', sep='\t', encoding='utf-8', header='true')
"""

"\nyelp_df_la.to_csv(r'../Database/temp_csv/Los_Angeles.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_sd.to_csv(r'../Database/temp_csv/San_Diego.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_sj.to_csv(r'../Database/temp_csv/San_Jose.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_sf.to_csv(r'../Database/temp_csv/San_Francisco.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_fr.to_csv(r'../Database/temp_csv/Fresno.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_sc.to_csv(r'../Database/temp_csv/Sacramento.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_lb.to_csv(r'../Database/temp_csv/Long_Beach.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_ok.to_csv(r'../Database/temp_csv/Oakland.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_bf.to_csv(r'../Database/temp_csv/Bakersfield.csv', sep='\t', encoding='utf-8', header='true')\nyelp_df_ah.to_csv(r'../Database/temp_csv/Anaheim.csv', sep='\t', encoding='utf-8', h

### Data concatenation

In [ ]:
# 
def importer(start_index):
    pathlist = Path('data').glob('**/*.csv')
    data_arr = []
    for path in pathlist:
        data_arr.append(str(path))
    data_arr.sort()
    
    if(start_index < 0 or start_index > len(data_arr)):
        print("Start index is invalid and importer will import all the csv files. ")
        start_index = 0
        
    final_data = []
    for data in data_arr[start_index:]:
        print("Reading the data file: " + data)
        frame = pd.read_csv(data)
        fully_paid = frame[frame['loan_status'] == 'Fully Paid']
        charged_off = frame[frame['loan_status'] == 'Charged Off']
        final_data.append(fully_paid)
        final_data.append(charged_off)
    
    dataset = pd.concat(final_data)
    return dataset

## Data analysis playground